In [2]:
import numpy as np
import clusters
import datasets as ds
import estimate_k as ek

from sklearn.cluster import SpectralClustering

In [4]:
# prepare for datasets

# mnist
mnist_train,_,mnist_test,mnist_test_lable = ds.get_mnist()
mnist_train = mnist_train.astype('float32') / 255.
mnist_train = mnist_train.reshape((len(mnist_train),np.prod(mnist_train.shape[1:]))) # make 28*28 img to 784 demission matrix
mnist_test = mnist_test.astype('float32') / 255.
mnist_test = mnist_test.reshape((len(mnist_test), np.prod(mnist_test.shape[1:])))

# fashion mnist
fashion_train,_,fashion_test,fashion_test_lable = ds.get_fashion_mnist()

# reuters_8
reuters_train,_, reuters_test,_ = ds.get_reuters_8()

In [2]:
def process(train,test,true_k):
    # DE
    de_result = clusters.de(train,test) # train result is based on 'test', so clustering score should base on 'test' too
    print(de_result.shape)

    # SA
    k = ek.get_k_from_result(20,ek.estimate_mnist(20, de_result,test))
    print("estimate k :")
    print(k)

    # SC clustering
    sc = SpectralClustering(n_clusters=true_k, eigen_solver='arpack', affinity="nearest_neighbors")
    sc_result = sc.fit_predict(test)
    score = clusters.score(sc_result)
    print("sc score with true k:")
    print(score)

    # SCDE clustering
    sc = SpectralClustering(n_clusters=true_k, eigen_solver='arpack', affinity="nearest_neighbors")
    scde_result = sc.fit_predict(de_result)
    score = clusters.score(scde_result,test)
    print("scde score with true k:")
    print(score)

    return